## 수도 마스크 생성

In [7]:
import os
import torch
import numpy as np
import pydicom
import SimpleITK as sitk
from scipy.ndimage import zoom
from tqdm import tqdm
import segmentation_models_pytorch as smp
import torch.nn.functional as F

# --- [1. 경로 및 설정] ---
TRAIN_IMAGES_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train_images'
WEIGHTS_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\checkpoints' 
MASK_SAVE_ROOT = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_segmentation_masks_2019'

TARGET_Z, TARGET_SIZE = 128, 224
WL, WW = 500, 2000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(MASK_SAVE_ROOT, exist_ok=True)

# --- [2. 모델 5개 로드] ---
models = []
print("🔄 5-Fold Seg 모델 가중치를 불러오는 중...")
for f in range(1, 6):
    model_path = os.path.join(WEIGHTS_DIR, f'best_model_fold{f}.pth')
    m = smp.Unet(encoder_name="resnet34", in_channels=2, classes=8).to(DEVICE)
    m.load_state_dict(torch.load(model_path, map_location=DEVICE))
    m.eval()
    models.append(m)
print(f"✅ 5-Fold 모델 앙상블 준비 완료")

# --- [3. 유틸리티 함수] ---
def apply_windowing(img, wl, ww):
    img_min, img_max = wl - ww // 2, wl + ww // 2
    img = np.clip(img, img_min, img_max)
    return ((img - img_min) / ww).astype(np.float32)

# --- [4. 빵꾸난 마스크 채우기 루프] ---
@torch.no_grad()
def run_ensemble_factory_fill_missing():
    all_uids = [d for d in os.listdir(TRAIN_IMAGES_DIR) if os.path.isdir(os.path.join(TRAIN_IMAGES_DIR, d))]
    
    # 🎯 [핵심] 루프를 돌기 전에, 마스크가 없는 환자만 미리 골라냅니다!
    missing_uids = []
    for uid in all_uids:
        if not os.path.exists(os.path.join(MASK_SAVE_ROOT, f"{uid}.nii.gz")):
            missing_uids.append(uid)

    print(f"\n📦 전체 환자: {len(all_uids)}명")
    print(f"✅ 이미 마스크가 있는 환자: {len(all_uids) - len(missing_uids)}명")
    print(f"🔥 새로 만들어야 할 환자: {len(missing_uids)}명")
    
    if len(missing_uids) == 0:
        print("🎉 모든 환자의 마스크가 이미 완벽하게 존재합니다! 더 이상 할 작업이 없습니다.")
        return

    # 이제 빵꾸난 환자들만 루프를 돕니다.
    for uid in tqdm(missing_uids, desc="Generating Missing Masks"):
        save_path = os.path.join(MASK_SAVE_ROOT, f"{uid}.nii.gz") 

        try:
            dicom_dir = os.path.join(TRAIN_IMAGES_DIR, uid)
            dcm_files = [f for f in os.listdir(dicom_dir) if f.endswith('.dcm')] 
            
            if len(dcm_files) == 0:
                print(f"⚠️ {uid} 폴더에 DICOM 파일이 없습니다. (혹시 이미 PNG로 변환하고 지우셨나요?)")
                continue
                
            dcm_data = []
            for f in dcm_files:
                ds = pydicom.dcmread(os.path.join(dicom_dir, f))
                dcm_data.append({
                    'pixel': ds.pixel_array.astype(np.float32) * float(getattr(ds, 'RescaleSlope', 1)) + float(getattr(ds, 'RescaleIntercept', 0)),
                    'z_pos': float(ds.ImagePositionPatient[2])
                })
            
            dcm_data.sort(key=lambda x: x['z_pos'], reverse=True)
            orig_vol = np.stack([x['pixel'] for x in dcm_data], axis=0)
            orig_z, orig_h, orig_w = orig_vol.shape

            vol_win = apply_windowing(orig_vol, WL, WW)
            input_vol = zoom(vol_win, (TARGET_Z/orig_z, TARGET_SIZE/orig_h, TARGET_SIZE/orig_w), order=1)

            ensemble_probs = np.zeros((TARGET_Z, 8, TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

            for z in range(TARGET_Z):
                img_slice = torch.from_numpy(input_vol[z]).unsqueeze(0)
                z_pos = torch.full((1, TARGET_SIZE, TARGET_SIZE), z / (TARGET_Z - 1))
                input_tensor = torch.cat([img_slice, z_pos], dim=0).unsqueeze(0).to(DEVICE)

                combined_prob = torch.zeros((1, 8, TARGET_SIZE, TARGET_SIZE)).to(DEVICE)
                for m in models:
                    output = m(input_tensor)
                    combined_prob += F.softmax(output, dim=1)
                
                ensemble_probs[z] = (combined_prob / 5.0).cpu().numpy()[0]

            mask_128 = np.argmax(ensemble_probs, axis=1).astype(np.uint8)

            full_mask = zoom(mask_128, (orig_z/TARGET_Z, orig_h/TARGET_SIZE, orig_w/TARGET_SIZE), order=0)
            full_mask = np.flip(full_mask, axis=(0, 1)) 

            mask_img = sitk.GetImageFromArray(full_mask)
            sitk.WriteImage(mask_img, save_path)

        except Exception as e:
            print(f"❌ {uid} 처리 실패: {e}")

if __name__ == '__main__':
    run_ensemble_factory_fill_missing()

🔄 5-Fold Seg 모델 가중치를 불러오는 중...
✅ 5-Fold 모델 앙상블 준비 완료

📦 전체 환자: 2019명
✅ 이미 마스크가 있는 환자: 1328명
🔥 새로 만들어야 할 환자: 691명


Generating Missing Masks: 100%|██████████| 691/691 [1:16:35<00:00,  6.65s/it]


## 가짜 정답지 생성

In [8]:
import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm

# --- [1. 경로 설정] ---
# 주최측이 준 2019명 원본 정답지
TRAIN_CSV_PATH = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train.csv'

# Step 1에서 우리가 뽑아낸 2019명 NIfTI 마스크 폴더
MASK_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_segmentation_masks_2019'

# 🔥 새롭게 탄생할 60만 장짜리 거대 가짜 정답지 저장 경로
SAVE_CSV_PATH = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_slice_labels_2019.csv'

print("🔄 1. 환자별 정답지(train.csv) 로딩 중...")
df_train = pd.read_csv(TRAIN_CSV_PATH)

# 환자 UID를 넣으면 -> C1~C7 골절 여부를 바로 알려주는 사전(Dictionary) 만들기
uid_to_labels = {}
for _, row in df_train.iterrows():
    uid_to_labels[row['StudyInstanceUID']] = {
        1: row['C1'], 2: row['C2'], 3: row['C3'],
        4: row['C4'], 5: row['C5'], 6: row['C6'], 7: row['C7']
    }

nifti_files = [f for f in os.listdir(MASK_DIR) if f.endswith('.nii.gz')]
print(f"📁 총 {len(nifti_files)}명의 마스크 파일을 분석합니다.")

results = []

# --- [2. 마스크를 훑으며 슬라이스별 정답 생성] ---
for f in tqdm(nifti_files, desc="🧬 슬라이스별 가짜 정답(Weak Label) 생성 중"):
    uid = f.replace('.nii.gz', '')

    # 만약 train.csv에 없는 환자면 패스
    if uid not in uid_to_labels:
        continue

    patient_labels = uid_to_labels[uid] # 이 환자의 뼈별 골절 정답 (예: C4=1, 나머지=0)

    # 3D 마스크 로드 (SimpleITK를 쓰면 Z축 슬라이스 순서가 완벽하게 유지됩니다)
    nifti_path = os.path.join(MASK_DIR, f)
    img = sitk.ReadImage(nifti_path)
    mask_array = sitk.GetArrayFromImage(img) # 형태: (Z_slices, Height, Width)

    num_slices = mask_array.shape[0]

    for z in range(num_slices):
        slice_mask = mask_array[z]
        
        # 이 슬라이스 사진 안에 도대체 몇 번 뼈들이 들어있나 확인
        unique_bones = np.unique(slice_mask)
        # 배경(0)이나 잡음 빼고 오직 1번~7번(C1~C7) 뼈만 남김
        unique_bones = [b for b in unique_bones if 1 <= b <= 7]

        # 🎯 [핵심] 뼈가 단 하나도 안 찍힌 정수리, 가슴팍 사진은 학습 데이터에서 쿨하게 버림!
        if len(unique_bones) == 0:
            continue

        # 골절 여부 판별 로직
        slice_fracture = 0
        for bone in unique_bones:
            # 이 슬라이스에 찍힌 뼈 중에, 단 하나라도 train.csv에서 '골절'이라고 한 뼈가 있다면?
            if patient_labels[bone] == 1:
                slice_fracture = 1  # 이 슬라이스는 무조건 "골절(1)" 라벨 부여!
                break

        # 보통 NIfTI Z축 인덱스(0부터) + 1 = 원본 DICOM의 슬라이스 번호(InstanceNumber)
        slice_number = z + 1 

        # 나중에 확인하기 편하게 "이 사진엔 C2, C3가 찍혀있다"는 메모도 같이 남깁니다.
        bones_str = "_".join([f"C{b}" for b in unique_bones])

        # 결과 저장
        results.append({
            'StudyInstanceUID': uid,
            'slice_number': slice_number,
            'present_bones': bones_str,
            'fracture': slice_fracture
        })

# --- [3. 결과물 CSV 저장] ---
df_results = pd.DataFrame(results)
df_results.to_csv(SAVE_CSV_PATH, index=False)

print("\n✨ [완료] 약지도 학습(Weak Labeling)용 마스터 정답지가 완성되었습니다!")
print(f"📊 최종 확보된 알짜배기 뼈 슬라이스 개수: {len(df_results):,}장")
print(f"💾 저장 경로: {SAVE_CSV_PATH}")

# 결과 미리보기
display(df_results.head(10))

🔄 1. 환자별 정답지(train.csv) 로딩 중...
📁 총 2019명의 마스크 파일을 분석합니다.


🧬 슬라이스별 가짜 정답(Weak Label) 생성 중: 100%|██████████| 2019/2019 [07:00<00:00,  4.80it/s]



✨ [완료] 약지도 학습(Weak Labeling)용 마스터 정답지가 완성되었습니다!
📊 최종 확보된 알짜배기 뼈 슬라이스 개수: 448,032장
💾 저장 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_slice_labels_2019.csv


,StudyInstanceUID,slice_number,present_bones,fracture
0,1.2.826.0.1.3680043.10001,36,C7,0
1,1.2.826.0.1.3680043.10001,37,C7,0
2,1.2.826.0.1.3680043.10001,40,C7,0
3,1.2.826.0.1.3680043.10001,41,C7,0
4,1.2.826.0.1.3680043.10001,42,C7,0
5,1.2.826.0.1.3680043.10001,43,C7,0
6,1.2.826.0.1.3680043.10001,44,C7,0
7,1.2.826.0.1.3680043.10001,45,C7,0
8,1.2.826.0.1.3680043.10001,46,C7,0
9,1.2.826.0.1.3680043.10001,47,C7,0


## dicom -> png

In [9]:
import os
import pydicom
import numpy as np
import cv2
from tqdm import tqdm
import re

# ==========================================
# [설정] 경로 및 파라미터
# ==========================================
# 2019명 환자의 원본 DICOM이 들어있는 최상위 폴더
TRAIN_IMAGES_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train_images'

# 뼈를 선명하게 보기 위한 Windowing 설정 (절대 고정!)
WL, WW = 500, 2000

# ==========================================
# [함수] 윈도윙 처리
# ==========================================
def apply_windowing(img, wl, ww):
    img_min, img_max = wl - ww // 2, wl + ww // 2
    img = np.clip(img, img_min, img_max)
    # 0.0 ~ 1.0 사이의 float 값으로 반환
    return ((img - img_min) / ww).astype(np.float32)

def get_slice_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

# ==========================================
# [메인] 일괄 변환 및 즉시 삭제 로직
# ==========================================
def convert_and_destroy():
    if not os.path.exists(TRAIN_IMAGES_DIR):
        print(f"❌ 경로를 찾을 수 없습니다: {TRAIN_IMAGES_DIR}")
        return

    uids = [d for d in os.listdir(TRAIN_IMAGES_DIR) if os.path.isdir(os.path.join(TRAIN_IMAGES_DIR, d))]
    print(f"🚀 [DICOM -> PNG 변환기 가동] 총 {len(uids)}명 환자 처리 시작!")
    print("⚠️ 경고: 변환이 완료된 .dcm 파일은 하드디스크에서 즉시 영구 삭제됩니다!")

    # 성공, 실패 카운트
    total_converted = 0
    total_deleted = 0

    for uid in tqdm(uids, desc="환자 단위 처리 중"):
        patient_dir = os.path.join(TRAIN_IMAGES_DIR, uid)
        
        # 이 환자 폴더 안의 모든 파일 가져오기
        files = os.listdir(patient_dir)
        
        # .dcm 파일만 필터링
        dcm_files = [f for f in files if f.endswith('.dcm')]
        
        for dcm_filename in dcm_files:
            dcm_path = os.path.join(patient_dir, dcm_filename)
            
            # 새롭게 저장될 PNG 파일 이름 설정 (확장자만 .png로 변경)
            png_filename = dcm_filename.replace('.dcm', '.png')
            png_path = os.path.join(patient_dir, png_filename)
            
            # 이미 변환된 PNG가 있다면 굳이 다시 안 만들고 DICOM만 지움 (이어하기 방어 로직)
            if os.path.exists(png_path):
                try:
                    os.remove(dcm_path)
                    total_deleted += 1
                except: pass
                continue

            try:
                # 1. DICOM 읽기
                ds = pydicom.dcmread(dcm_path)
                
                # 2. 픽셀값 복원 (Slope & Intercept 적용)
                slope = float(getattr(ds, 'RescaleSlope', 1.0))
                intercept = float(getattr(ds, 'RescaleIntercept', 0.0))
                px = ds.pixel_array.astype(np.float32) * slope + intercept
                
                # 3. 뼈 윈도윙 적용 (0.0 ~ 1.0)
                img_float = apply_windowing(px, WL, WW)
                
                # 4. 이미지 저장을 위해 0~255 (uint8) 형태로 변환
                img_uint8 = (img_float * 255.0).astype(np.uint8)
                
                # 5. PNG 저장 (압축률 최대로 가볍게!)
                success = cv2.imwrite(png_path, img_uint8)
                
                # 6. 🗑️ 저장이 완벽하게 성공했을 때만 DICOM 원본 파기!
                if success:
                    os.remove(dcm_path)
                    total_converted += 1
                    total_deleted += 1
                    
            except Exception as e:
                # 파일이 깨졌거나 문제가 생기면 스킵 (지우지 않음)
                print(f"❌ 변환 에러 [{uid} / {dcm_filename}]: {e}")
                continue

    print(f"\n✨ [작업 완료] 대청소가 끝났습니다!")
    print(f"   - 🖼️ 새롭게 생성된 PNG: {total_converted:,}장")
    print(f"   - 🗑️ 영구 삭제된 DICOM: {total_deleted:,}장")
    print(f"   -> 하드디스크 용량을 다시 확인해 보세요! 엄청나게 쾌적해졌을 겁니다!")

if __name__ == '__main__':
    convert_and_destroy()

🚀 [DICOM -> PNG 변환기 가동] 총 2019명 환자 처리 시작!
⚠️ 경고: 변환이 완료된 .dcm 파일은 하드디스크에서 즉시 영구 삭제됩니다!


환자 단위 처리 중: 100%|██████████| 2019/2019 [2:17:57<00:00,  4.10s/it]  


✨ [작업 완료] 대청소가 끝났습니다!
   - 🖼️ 새롭게 생성된 PNG: 711,601장
   - 🗑️ 영구 삭제된 DICOM: 711,601장
   -> 하드디스크 용량을 다시 확인해 보세요! 엄청나게 쾌적해졌을 겁니다!


## 전처리

In [11]:
import os
import numpy as np
import SimpleITK as sitk
import cv2
import pandas as pd
from tqdm import tqdm
import re
import gc

# ==========================================
# [설정] 경로 및 파라미터
# ==========================================
# 변환된 PNG 파일들이 있는 폴더
TRAIN_IMAGES_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train_images'
MASK_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_segmentation_masks_2019'
PSEUDO_CSV_PATH = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\pseudo_slice_labels_2019.csv'

# 🔥 3채널 크롭 이미지가 저장될 새로운 목적지 (폴더명 변경)
OUTPUT_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\detection_cropped_2019_512_3ch'
OUTPUT_MANIFEST_FULL = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv'

TARGET_SIZE = 512
MARGIN_RATIO = 0.2 

# ==========================================
# [함수] 유틸리티
# ==========================================
def get_slice_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

def get_mask_bbox(mask_slice):
    rows = np.any(mask_slice > 0, axis=1)
    cols = np.any(mask_slice > 0, axis=0)
    if not np.any(rows) or not np.any(cols): return None
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return x_min, y_min, x_max, y_max

def make_square_box(x1, y1, x2, y2, img_w, img_h, margin=0.2):
    w = x2 - x1
    h = y2 - y1
    cx, cy = x1 + w // 2, y1 + h // 2
    max_side = max(w, h)
    new_size = int(max_side * (1 + margin))
    half_size = new_size // 2
    new_x1 = max(0, cx - half_size)
    new_y1 = max(0, cy - half_size)
    new_x2 = min(img_w, cx + half_size)
    new_y2 = min(img_h, cy + half_size)
    if new_x2 <= new_x1 or new_y2 <= new_y1: return 0, 0, img_w, img_h
    return int(new_x1), int(new_y1), int(new_x2), int(new_y2)

# ==========================================
# [메인] 3채널 쾌속 크롭 (이어하기 완벽 지원)
# ==========================================
def process_png_crop_3ch():
    if not os.path.exists(PSEUDO_CSV_PATH):
        print(f"❌ 가짜 정답지 CSV가 없습니다.")
        return
    
    df_pseudo = pd.read_csv(PSEUDO_CSV_PATH)
    print(f"📖 가짜 정답지 로드 완료: {len(df_pseudo):,}장 크롭 대기 중")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    uids_to_process = df_pseudo['StudyInstanceUID'].unique()
    new_rows = []
    
    print(f"🚀 [Speed 3-Channel Crop] 2019명 3채널 크롭 시작!")

    for uid in tqdm(uids_to_process):
        patient_dir = os.path.join(TRAIN_IMAGES_DIR, uid)
        if not os.path.exists(patient_dir): continue 

        patient_slices = df_pseudo[df_pseudo['StudyInstanceUID'] == uid]
        target_slice_numbers = set(patient_slices['slice_number'].values)
        
        mask_path = os.path.join(MASK_DIR, f"{uid}.nii.gz")
        patient_save_dir = os.path.join(OUTPUT_DIR, uid)
        os.makedirs(patient_save_dir, exist_ok=True)

        try:
            png_files = [f for f in os.listdir(patient_dir) if f.endswith('.png')]
            png_meta = []
            for f in png_files:
                png_meta.append({
                    'full_path': os.path.join(patient_dir, f),
                    'slice_number': get_slice_number(f)
                })
            png_meta.sort(key=lambda x: x['slice_number'])
            
            target_z_indices = []
            for z_idx, meta in enumerate(png_meta):
                if meta['slice_number'] in target_slice_numbers:
                    target_z_indices.append((z_idx, meta))
            
            needs_processing = False
            for z_idx, meta in target_z_indices:
                if not os.path.exists(os.path.join(patient_save_dir, f"{z_idx:04d}.npz")):
                    needs_processing = True
                    break
            
            if needs_processing:
                if not os.path.exists(mask_path): continue 
                
                # 3D 마스크 로드
                mask_itk = sitk.ReadImage(mask_path)
                mask_vol = sitk.GetArrayFromImage(mask_itk).astype(np.uint8)

                # 🔥 PNG 이미지가 가벼우므로 한 환자 전체를 RAM에 한 번에 올립니다! (속도 극대화)
                full_vol = []
                for meta in png_meta:
                    img = cv2.imread(meta['full_path'], cv2.IMREAD_GRAYSCALE)
                    full_vol.append(img)
                full_vol = np.stack(full_vol, axis=0) # (Z, H, W)

                for z_idx, meta in target_z_indices:
                    save_path = os.path.join(patient_save_dir, f"{z_idx:04d}.npz")
                    if os.path.exists(save_path): continue 

                    if z_idx >= len(mask_vol): break
                    m_slice = mask_vol[z_idx]
                    
                    mask_bbox = get_mask_bbox(m_slice)
                    if mask_bbox is None: continue
                    mx1, my1, mx2, my2 = mask_bbox
                    
                    ori_h, ori_w = m_slice.shape
                    cx1, cy1, cx2, cy2 = make_square_box(mx1, my1, mx2, my2, ori_w, ori_h, MARGIN_RATIO)
                    crop_w, crop_h = cx2 - cx1, cy2 - cy1
                    if crop_w == 0: continue
                    
                    # 🎯 [핵심] 3채널 묶기 로직 부활!
                    z_prev = max(0, z_idx - 1)
                    z_next = min(len(full_vol) - 1, z_idx + 1)
                    img_3ch = np.stack([full_vol[z_prev], full_vol[z_idx], full_vol[z_next]], axis=0) 
                    
                    # 크롭 및 리사이즈
                    cropped_3ch = img_3ch[:, cy1:cy2, cx1:cx2]
                    cropped_hwc = np.transpose(cropped_3ch, (1, 2, 0)) # OpenCV 리사이즈를 위해 HWC 변환
                    resized_hwc = cv2.resize(cropped_hwc, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR)
                    final_data = np.transpose(resized_hwc, (2, 0, 1)) # 다시 파이토치가 좋아하는 CHW 변환
                    
                    np.savez_compressed(save_path, data=final_data)
                
                del mask_vol, full_vol
                gc.collect()

            for z_idx, meta in target_z_indices:
                save_path = os.path.join(patient_save_dir, f"{z_idx:04d}.npz")
                if os.path.exists(save_path):
                    current_slice_num = meta['slice_number']
                    is_fracture = patient_slices[patient_slices['slice_number'] == current_slice_num]['fracture'].values[0]
                    present_bones = patient_slices[patient_slices['slice_number'] == current_slice_num]['present_bones'].values[0]

                    new_rows.append({
                        'StudyInstanceUID': uid,
                        'slice_idx': z_idx,
                        'original_slice_number': current_slice_num,
                        'file_path': os.path.join('detection_cropped_2019_512_3ch', uid, f"{z_idx:04d}.npz"), # 폴더명 주의!
                        'fracture': is_fracture,
                        'present_bones': present_bones
                    })
                
        except Exception as e:
            print(f"❌ Error {uid}: {e}")
            continue

    full_df = pd.DataFrame(new_rows)
    full_df.to_csv(OUTPUT_MANIFEST_FULL, index=False)
    
    print(f"\n✅ [Complete] 2019명 3채널 크롭 완료!")
    print(f"   - 총 매니페스트 생성 수: {len(full_df):,}장")
    print(f"   - 최종 매니페스트 저장: {OUTPUT_MANIFEST_FULL}")

if __name__ == '__main__':
    process_png_crop_3ch()

📖 가짜 정답지 로드 완료: 448,032장 크롭 대기 중
🚀 [Speed 3-Channel Crop] 2019명 3채널 크롭 시작!


100%|██████████| 2019/2019 [4:15:53<00:00,  7.60s/it]  



✅ [Complete] 2019명 3채널 크롭 완료!
   - 총 매니페스트 생성 수: 447,958장
   - 최종 매니페스트 저장: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv


## 학습

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import timm  
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GroupShuffleSplit
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt 

# --- [1. 시스템 에러 및 멈춤 방지] ---
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
cv2.setNumThreads(0) 

# ==========================================
# 2. 설정 (Configuration)
# ==========================================
CONFIG = {
    'base_dir': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection',
    'manifest_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv',
    
    # 🛡️ 235명 격리용 Golden Box CSV
    'golden_bbox_csv': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\filtered_train_bounding_boxes.csv',
    
    'model_name': 'tf_efficientnetv2_s.in21k_ft_in1k', 
    'img_size': 512,    
    'batch_size': 16,   
    'epochs': 50,
    'patience': 10,       
    'learning_rate': 5e-5, 
    'weight_decay': 5e-2,  
    
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'save_dir': './runs/rsna_clf_efficientnet_pseudo_2019'
}

# ==========================================
# 3. 데이터 증강 (Albumentations) - 의료 영상 최적화 (최신 버전 반영)
# ==========================================
train_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.HorizontalFlip(p=0.5), 
    # ShiftScaleRotate 대신 최신 함수인 Affine 사용
    A.Affine(scale=(0.95, 1.05), translate_percent=(-0.05, 0.05), rotate=(-5, 5), p=0.5), 
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5), 
    # GaussNoise의 옛날 파라미터(var_limit) 제거하고 기본값 사용
    A.GaussNoise(p=0.2), 
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2()
])

val_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ==========================================
# 4. 데이터셋 (초고속 정적 3채널 로딩)
# ==========================================
class RSNAClfDataset(Dataset):
    def __init__(self, df, base_dir, transforms=None):
        self.df = df
        self.base_dir = base_dir
        self.transforms = transforms
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # 'old_train' 폴더 안의 'detection_cropped_2019_512_3ch' 경로를 정확히 잡아줍니다.
        full_path = os.path.join(self.base_dir, 'old_train', row['file_path'])
        
        # 1. 만들어둔 3채널 npz 단숨에 로드!
        try:
            with np.load(full_path) as loaded:
                img = loaded['data'].astype(np.float32) # Shape: (3, 512, 512)
        except:
            img = np.zeros((3, 512, 512), dtype=np.float32)

        # 2. 각 채널별로 CLAHE (명암 대비 극대화) 적용
        img_uint8 = (img * 255).astype(np.uint8)
        channels = []
        for i in range(3):
            channels.append(self.clahe.apply(img_uint8[i]))
        
        # 3. Albumentations 증강을 위해 HWC 형태로 결합
        img_hwc = np.stack(channels, axis=-1) 
        
        if self.transforms:
            augmented = self.transforms(image=img_hwc)
            img_tensor = augmented['image']
        else:
            img_tensor = torch.from_numpy(np.transpose(img_hwc, (2,0,1))).float() / 255.0

        label = torch.tensor([float(row['fracture'])], dtype=torch.float32)
        return img_tensor, label

# ==========================================
# 5. 모델 구성
# ==========================================
def build_model(model_name, pretrained=True):
    print(f"🛠️ 모델 빌드 중: {model_name}")
    model = timm.create_model(
        model_name, pretrained=pretrained, in_chans=3, num_classes=1,
        drop_rate=0.3, drop_path_rate=0.2 
    )
    return model

# ==========================================
# 6. 학습 및 검증 함수
# ==========================================
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(dataloader, desc="Training", leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
    return running_loss / len(dataloader)

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    pbar = tqdm(dataloader, desc="Validating", leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        
        probs = torch.sigmoid(outputs).cpu().numpy()
        targets = labels.cpu().numpy()
        
        all_preds.extend(probs)
        all_targets.extend(targets)
        
    all_preds = np.array(all_preds).flatten()
    all_targets = np.array(all_targets).flatten()
    preds_binary = (all_preds >= 0.5).astype(int)
    
    val_loss = running_loss / len(dataloader)
    
    try: auc = roc_auc_score(all_targets, all_preds)
    except: auc = 0.5 
    
    acc = accuracy_score(all_targets, preds_binary)
    recall = recall_score(all_targets, preds_binary, zero_division=0)
    precision = precision_score(all_targets, preds_binary, zero_division=0)
    f1 = f1_score(all_targets, preds_binary, zero_division=0)
    
    return val_loss, auc, acc, recall, precision, f1

# ==========================================
# 7. 메인 실행 
# ==========================================
if __name__ == '__main__':
    os.makedirs(CONFIG['save_dir'], exist_ok=True)
    
    # 1. 매니페스트 로드
    df_full = pd.read_csv(CONFIG['manifest_path'])
    
    # 2. 🛡️ 235명(Golden Test Set) 완벽 격리!
    df_golden = pd.read_csv(CONFIG['golden_bbox_csv'])
    golden_uids = set(df_golden['StudyInstanceUID'].astype(str).unique())
    df_full['StudyInstanceUID'] = df_full['StudyInstanceUID'].astype(str)
    
    df_train_val = df_full[~df_full['StudyInstanceUID'].isin(golden_uids)].reset_index(drop=True)
    
    print(f"📦 전체 데이터 환자 수: {df_full['StudyInstanceUID'].nunique()}명")
    print(f"🛡️ 테스트용으로 제외된 환자 수 (Golden): {len(golden_uids)}명")
    print(f"🚀 학습/검증에 사용할 환자 수: {df_train_val['StudyInstanceUID'].nunique()}명")

    # 3. 환자 단위로 8:2 스플릿 
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(gss.split(df_train_val, groups=df_train_val['StudyInstanceUID']))
    
    train_df = df_train_val.iloc[train_idx].reset_index(drop=True)
    val_df = df_train_val.iloc[val_idx].reset_index(drop=True)
    
    # 4. ⚖️ Train 데이터 1:1 강제 언더샘플링!
    pos_df = train_df[train_df['fracture'] == 1]
    neg_df = train_df[train_df['fracture'] == 0]
    
    if len(neg_df) > len(pos_df):
        neg_sampled = neg_df.sample(n=len(pos_df), random_state=42)
    else:
        neg_sampled = neg_df
        
    train_df_balanced = pd.concat([pos_df, neg_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("\n⚖️ [데이터 분할 완료]")
    print(f"  - Train (1:1 밸런스): {len(train_df_balanced)}장 (골절 {len(pos_df)} / 정상 {len(neg_sampled)})")
    print(f"  - Val (실제 비율 유지): {len(val_df)}장 (골절 {len(val_df[val_df['fracture']==1])} / 정상 {len(val_df[val_df['fracture']==0])})")

    # 5. Dataset 및 DataLoader 준비
    train_ds = RSNAClfDataset(train_df_balanced, CONFIG['base_dir'], transforms=train_transforms)
    val_ds = RSNAClfDataset(val_df, CONFIG['base_dir'], transforms=val_transforms)
    
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0, pin_memory=True)
    
    device = CONFIG['device']
    model = build_model(CONFIG['model_name']).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])
    
    best_auc = 0.0
    early_stop_counter = 0 
    
    history = {
        'epoch': [], 'train_loss': [], 'val_loss': [], 
        'val_auc': [], 'val_acc': [], 'val_recall': [],
        'val_precision': [], 'val_f1': []
    }
    
    print(f"\n🚀 [Classification Mode] 1784명 환자 거대 학습 시작! (Model: {CONFIG['model_name']})")
    
    for epoch in range(1, CONFIG['epochs'] + 1):
        print(f"\n[Epoch {epoch}/{CONFIG['epochs']}]")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_auc, val_acc, val_recall, val_precision, val_f1 = validate(model, val_loader, criterion, device)
        
        scheduler.step() 
        
        print(f"📉 Train Loss: {train_loss:.4f} | 📈 Val Loss: {val_loss:.4f}")
        print(f"🏆 AUC: {val_auc:.4f} | Acc: {val_acc:.4f} | 🔥 Recall: {val_recall:.4f} | 🎯 Prec: {val_precision:.4f} | ⚖️ F1: {val_f1:.4f}")
        
        history['epoch'].append(epoch)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_auc'].append(val_auc)
        history['val_acc'].append(val_acc)
        history['val_recall'].append(val_recall)
        history['val_precision'].append(val_precision)
        history['val_f1'].append(val_f1)

        # ==========================================================
        # 🔥 [여기 추가!] 매 에폭마다 즉시 엑셀(CSV) 덮어쓰기 저장 (자동 저장)
        # ==========================================================
        history_df = pd.DataFrame(history)
        csv_path = os.path.join(CONFIG['save_dir'], "history_pseudo_2019.csv")
        history_df.to_csv(csv_path, index=False)
        # ==========================================================
        
        if val_auc > best_auc:
            best_auc = val_auc
            early_stop_counter = 0 
            save_path = os.path.join(CONFIG['save_dir'], "best_efficientnet_pseudo_2019.pth")
            torch.save(model.state_dict(), save_path)
            print(f"💾 Best Model 저장됨! (AUC: {best_auc:.4f}) - 카운터 초기화 🔄")
        else:
            early_stop_counter += 1 
            print(f"⚠️ 기록 갱신 실패... (Early Stopping 카운터: {early_stop_counter}/{CONFIG['patience']})")
            
            if early_stop_counter >= CONFIG['patience']:
                print(f"\n🚨 {CONFIG['patience']} 에폭 동안 성능 향상이 없어 학습을 조기 종료(Early Stopping) 합니다!")
                break 
                
    print(f"\n✨ 거대 데이터 훈련 완료! 최고 AUC: {best_auc:.4f}")
    

    plt.figure(figsize=(15, 5)) 
    
    plt.subplot(1, 3, 1)
    plt.plot(history['epoch'], history['train_loss'], label='Train Loss', marker='o')
    plt.plot(history['epoch'], history['val_loss'], label='Val Loss', marker='o')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.subplot(1, 3, 2)
    plt.plot(history['epoch'], history['val_auc'], label='Val AUC', marker='o', color='red')
    plt.plot(history['epoch'], history['val_acc'], label='Val Acc', marker='o', color='purple')
    plt.title('AUC & Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.subplot(1, 3, 3)
    plt.plot(history['epoch'], history['val_recall'], label='Recall', marker='o', color='orange')
    plt.plot(history['epoch'], history['val_precision'], label='Precision', marker='o', color='green')
    plt.plot(history['epoch'], history['val_f1'], label='F1-Score', marker='o', color='blue')
    plt.title('Recall / Prec / F1')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    plot_path = os.path.join(CONFIG['save_dir'], "learning_curve_pseudo_2019.png")
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300)
    print(f"📈 그래프 이미지 저장 완료: {plot_path}")
    plt.show()

📦 전체 데이터 환자 수: 2019명
🛡️ 테스트용으로 제외된 환자 수 (Golden): 235명
🚀 학습/검증에 사용할 환자 수: 1784명

⚖️ [데이터 분할 완료]
  - Train (1:1 밸런스): 85260장 (골절 42630 / 정상 42630)
  - Val (실제 비율 유지): 82868장 (골절 8957 / 정상 73911)
🛠️ 모델 빌드 중: tf_efficientnetv2_s.in21k_ft_in1k

🚀 [Classification Mode] 1784명 환자 거대 학습 시작! (Model: tf_efficientnetv2_s.in21k_ft_in1k)

[Epoch 1/50]


📉 Train Loss: 0.9908 | 📈 Val Loss: 0.6985
🏆 AUC: 0.6314 | Acc: 0.5672 | 🔥 Recall: 0.6260 | 🎯 Prec: 0.1471 | ⚖️ F1: 0.2382
💾 Best Model 저장됨! (AUC: 0.6314) - 카운터 초기화 🔄

[Epoch 2/50]


📉 Train Loss: 0.6709 | 📈 Val Loss: 0.6569
🏆 AUC: 0.6524 | Acc: 0.5940 | 🔥 Recall: 0.6442 | 🎯 Prec: 0.1593 | ⚖️ F1: 0.2554
💾 Best Model 저장됨! (AUC: 0.6524) - 카운터 초기화 🔄

[Epoch 3/50]


📉 Train Loss: 0.5553 | 📈 Val Loss: 0.6864
🏆 AUC: 0.6355 | Acc: 0.6633 | 🔥 Recall: 0.5088 | 🎯 Prec: 0.1624 | ⚖️ F1: 0.2462
⚠️ 기록 갱신 실패... (Early Stopping 카운터: 1/10)

[Epoch 4/50]


Training:  98%|█████████▊| 5248/5329 [1:45:31<01:48,  1.34s/it, loss=0.7613]  

## 책갈피